In [1]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [3]:
number_of_tweets = 500
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [4]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-10 04:27:34,1267197215635726336,&lt;Ethereum Price&gt;\nETH/USDT(Binance): 318...
1,2022-02-10 04:27:14,1429172639629811712,"@elonmusk Its the first rule of politics, you ..."
2,2022-02-10 04:25:50,55264503,Tired of all the scary news? Need a break from...
3,2022-02-10 04:25:42,312761357,RT @kohidave: We have some EXECiting news ✨ Co...
4,2022-02-10 04:24:21,867117034089435136,"RT @WilliamBWorthy: ""The corrupt society thema..."
...,...,...,...
995,2022-02-09 19:03:07,54549682,RT @mirandadevine: Happy to report that my bes...
996,2022-02-09 19:02:26,1404406896220676099,GeForce RTX 3080 Ti - MSI - GAMING X TRIO is i...
997,2022-02-09 19:02:14,1385286231811051520,150 Pcs \n.\nONLY £1.99 \n.\nLOW STOCK \n.\nht...
998,2022-02-09 19:02:07,499672969,(US)In Stock: MSI Gaming RX 6800 - 1219.99\nAT...


In [5]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [6]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [7]:
df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets
0,2022-02-10 04:27:34,1267197215635726336,lt Ethereum Price gt ETH USDT Binance Check Pr...
1,2022-02-10 04:27:14,1429172639629811712,Its first rule politics control moron hate thi...
2,2022-02-10 04:25:50,55264503,Tired scary news Need break real world Need la...
3,2022-02-10 04:24:20,1267197215635726336,lt Ripple XRP Price gt XRP USDT Binance Check ...
4,2022-02-10 04:23:15,2320472316,How avoid Amazon Prime price hike per year month
...,...,...,...
393,2022-02-09 19:11:27,800289876948054016,Only I hear read APESNOTLEAVING talk stock goi...
394,2022-02-09 19:10:39,1444834958858543109,That would impact Dickbutts way good earnings ...
395,2022-02-09 19:10:15,634708561,We mostly talked company make product use Appl...
396,2022-02-09 19:05:14,402428798,Mega Rolls Regular Rolls Quilted Northern Ultr...


In [8]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Subjectivity,Polarity
0,2022-02-10 04:27:34,1267197215635726336,lt Ethereum Price gt ETH USDT Binance Check Pr...,0.000000,0.000000
1,2022-02-10 04:27:14,1429172639629811712,Its first rule politics control moron hate thi...,0.380208,-0.189583
2,2022-02-10 04:25:50,55264503,Tired scary news Need break real world Need la...,0.533333,-0.200000
3,2022-02-10 04:24:20,1267197215635726336,lt Ripple XRP Price gt XRP USDT Binance Check ...,0.000000,0.000000
4,2022-02-10 04:23:15,2320472316,How avoid Amazon Prime price hike per year month,0.000000,0.000000
...,...,...,...,...,...
393,2022-02-09 19:11:27,800289876948054016,Only I hear read APESNOTLEAVING talk stock goi...,0.658929,-0.066071
394,2022-02-09 19:10:39,1444834958858543109,That would impact Dickbutts way good earnings ...,0.600000,0.700000
395,2022-02-09 19:10:15,634708561,We mostly talked company make product use Appl...,0.433333,0.107292
396,2022-02-09 19:05:14,402428798,Mega Rolls Regular Rolls Quilted Northern Ultr...,0.025641,-0.022222


In [9]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
# matrix of n_grams
ngrams = c_vec.fit_transform(df['Tweets'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'Frequency', 1:'n-gram'})

df_ngram = df_ngram.drop_duplicates(subset='n-gram')
df_ngram = df_ngram.reset_index(drop=True)
df_ngram['Polarity'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).polarity)
df_ngram['Subjective'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).subjectivity)
df_ngram.drop(df_ngram.loc[df_ngram['Polarity'] == 0.0].index, inplace=True)
df_ngram = df_ngram.reset_index(drop=True)
df_ngram

,Frequency,n-gram,Polarity,Subjective
0,13,comic comicbooks news,0.250000,0.500000
1,13,comic comicbooks,0.250000,0.500000
2,8,new york,0.136364,0.454545
3,8,dccomics comic comicbooks,0.250000,0.500000
4,8,dccomics comic,0.250000,0.500000
...,...,...,...,...
1440,1,able get,0.500000,0.625000
1441,1,able far still,0.300000,0.812500
1442,1,able far,0.300000,0.812500
1443,1,able avoid obstacle,0.500000,0.625000


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv_vect = CountVectorizer(ngram_range=(2,3))
cv = cv_vect.fit_transform(df['Tweets'])


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

y = df['Polarity'].values.astype('float')

X_train, X_test, y_train, y_test = train_test_split(cv, y, random_state=42, test_size=0.2)
y_test


array([ 0.        , -0.02222222,  0.8       ,  0.        ,  0.        ,
        0.        , -0.375     ,  0.06818182,  0.        , -0.32222222,
        0.        ,  0.        , -0.11666667,  0.125     ,  0.01818182,
        0.1       ,  0.3       ,  0.13636364,  0.        , -0.15      ,
        0.        ,  0.31666667,  0.        ,  0.        ,  0.        ,
        0.7       , -0.04166667,  0.        ,  0.1       ,  0.        ,
       -0.06607143,  0.        ,  0.        ,  0.        ,  0.14285714,
        0.075     ,  0.1875    ,  0.        ,  0.25      , -1.        ,
        0.        ,  0.25      ,  0.        ,  0.07083333,  0.04545455,
        0.        ,  0.13636364,  0.25      ,  0.        ,  0.1875    ,
        0.125     ,  0.        ,  0.8       ,  1.        ,  0.36666667,
        0.05      ,  0.13636364, -0.16666667,  1.        ,  0.        ,
       -0.05      ,  0.35      ,  0.        , -0.18787879,  0.05      ,
        0.15416667,  0.42      ,  0.5       ,  0.        ,  0.13

In [13]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.LinearRegression()]

trainingData    = X_train
trainingScores  = y_train
predictionData  = X_test

for item in classifiers:
    print(item)
    clf = item
    clf.fit(trainingData, trainingScores)
    print(y_test)
    print(clf.predict(predictionData),'\n')

SVR()
[ 0.         -0.02222222  0.8         0.          0.          0.
 -0.375       0.06818182  0.         -0.32222222  0.          0.
 -0.11666667  0.125       0.01818182  0.1         0.3         0.13636364
  0.         -0.15        0.          0.31666667  0.          0.
  0.          0.7        -0.04166667  0.          0.1         0.
 -0.06607143  0.          0.          0.          0.14285714  0.075
  0.1875      0.          0.25       -1.          0.          0.25
  0.          0.07083333  0.04545455  0.          0.13636364  0.25
  0.          0.1875      0.125       0.          0.8         1.
  0.36666667  0.05        0.13636364 -0.16666667  1.          0.
 -0.05        0.35        0.         -0.18787879  0.05        0.15416667
  0.42        0.5         0.          0.13333333  0.06818182 -0.25833333
  0.          0.         -0.15625     0.          0.04        0.
  0.2         0.        ]
[ 0.13011674  0.14524297  0.11973678  0.12144889 -0.02138907  0.11581605
  0.11155736 -0.021